<a href="https://colab.research.google.com/github/jbmmbj/m494/blob/main/MNISTExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install FrEIA

  Preparing metadata (setup.py) ... done
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=42757 sha256=8ec65c986378bd097fed5b0b357c8dfaba3c0e76ae4851d23b4a9ad36ea5e1b2
  Stored in directory: /root/.cache/pip/wheels/81/a8/e2/d532a76f72108ac4a340cbe3f86b4f591abfdbd75209a5badb
Successfully built FrEIA


In [25]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import FrEIA.framework as Ff
import FrEIA.modules as Fm

# Define a simple invertible neural network using FrEIA
class InvertibleNN(nn.Module):
    def subnet_fc(self, in_features, out_features):
        hidden_size = self.hidden_size  # Access hidden_size from the parent class
        return nn.Sequential(
            nn.Linear(in_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_features),
            nn.Tanh()
        )

    def __init__(self, input_size, hidden_size, output_size):
        super(InvertibleNN, self).__init__()

        # Save hidden_size for later use in subnet_fc
        self.hidden_size = hidden_size

        # Build the architecture using FrEIA
        self.inn = Ff.SequenceINN(input_size)
        self.inn.append(Fm.AllInOneBlock, subnet_constructor=self.subnet_fc, permute_soft=True)
        self.inn.append(Fm.PermuteRandom)

        self.fc = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=1)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        # Forward pass through the invertible neural network
        x, _ = self.inn(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x

# Function to perform inverse pass through the model
def inverse_pass(model, output):
    # Invert the entire model
    inv_output, _ = model.inn(output, rev = True)

    return inv_output

# Load MNIST training data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

if __name__ == "__main__":
    input_size = 28 * 28  # MNIST image size
    hidden_size = 256
    output_size = 10

    model = InvertibleNN(input_size, hidden_size, output_size)

    # Training loop
    num_epochs = 5
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_data, batch_labels in train_loader:
            batch_data = batch_data.view(batch_data.size(0), -1)  # Flatten MNIST images
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)

            model.optimizer.zero_grad()
            loss.backward()
            model.optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Load MNIST test data
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2)

    # Test the model in the inverse direction
    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data = batch_data.view(batch_data.size(0), -1)  # Flatten MNIST images
            outputs = model(batch_data)
            inv_outputs = inverse_pass(model, outputs)

            print("Original Image:")
            print("Output from Forward Pass (Class Probabilities):", outputs)
            print("Reconstructed Image after Inverse Pass:")
            print("Inverse Output (Class Probabilities):", inv_outputs)
            break  # Break after processing the first batch for simplicity



RuntimeError: ignored

In [19]:
print(batch_data)

tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
         -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -0.8510, -0.1294,  0.2157,
          0.3647,  0.9922,  

In [21]:
outputs = model(batch_data)
outputs

tensor([[4.7770e-17, 4.4761e-18, 1.2245e-07, 1.0000e+00, 1.2594e-21, 1.0939e-13,
         1.9797e-17, 1.0807e-23, 1.1499e-17, 3.4565e-15]],
       grad_fn=<SoftmaxBackward0>)

In [31]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import FrEIA.framework as Ff
import FrEIA.modules as Fm

# Define a simple invertible neural network using FrEIA
class InvertibleNN(nn.Module):
    def subnet_fc(self, in_features, out_features):
        hidden_size = self.hidden_size  # Access hidden_size from the parent class
        return nn.Sequential(
            nn.Linear(in_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_features),
            nn.Tanh()
        )

    def __init__(self, input_size, hidden_size, output_size):
        super(InvertibleNN, self).__init__()

        # Save hidden_size for later use in subnet_fc
        self.hidden_size = hidden_size

        # Build the architecture using FrEIA
        self.inn = Ff.SequenceINN(input_size)

        self.inn.append(Fm.GLOWCouplingBlock, subnet_constructor = self.subnet_fc)

        self.inn.append(Fm.PermuteRandom)

        self.fc = nn.Linear(input_size, output_size)
        self.softmax = nn.Softmax(dim=1)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        # Forward pass through the invertible neural network
        x, _ = self.inn(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x


def inverse_pass(model, output):
    # Invert the entire model
    inv_output, _ = model(output, rev=True)

    return inv_output

# Load MNIST training data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

if __name__ == "__main__":
    input_size = 28 * 28  # MNIST image size
    hidden_size = 256
    output_size = 10

    model = InvertibleNN(input_size, hidden_size, output_size)

    # Training loop
    num_epochs = 5
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_data, batch_labels in train_loader:
            batch_data = batch_data.view(batch_data.size(0), -1)  # Flatten MNIST images
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)

            model.optimizer.zero_grad()
            loss.backward()
            model.optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Load MNIST test data
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=2)


    # Test the model in the inverse direction
    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data = batch_data.view(batch_data.size(0), -1)  # Flatten MNIST images
            outputs = model(batch_data)
            inv_outputs = inverse_pass(model, outputs)

            print("Original Image:")
            print("Output from Forward Pass (Class Probabilities):", outputs)
            print("Reconstructed Image after Inverse Pass:")
            print("Inverse Output (Class Probabilities):", inv_outputs)
            break  # Break after processing the first batch for simplicity


TypeError: ignored